In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
from tqdm.notebook import tqdm
import logging
import time

from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.inspection import permutation_importance


from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from bisect import bisect_left
from scipy.stats import rankdata
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

import optuna
import warnings
from pprint import pprint

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")

c:\Users\Adeniyi Babalola\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(
c:\Users\Adeniyi Babalola\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
RANDOM_STATE = 5
REPEATS = 3
sk = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

In [4]:
train = pd.read_csv(r'train.csv (1)\train.csv')
# test = pd.read_csv('../input/playground-series-s3e24/test.csv')

In [5]:
X = train.drop(['id', 'smoking'], axis=1)
y = train['smoking']

# X_test = test.drop('id', axis=1)

In [6]:
X.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,55,165,60,81.0,0.5,0.6,1,1,135,87,94,172,300,40,75,16.5,1,1.0,22,25,27,0
1,70,165,65,89.0,0.6,0.7,2,2,146,83,147,194,55,57,126,16.2,1,1.1,27,23,37,1
2,20,170,75,81.0,0.4,0.5,1,1,118,75,79,178,197,45,93,17.4,1,0.8,27,31,53,0
3,35,180,95,105.0,1.5,1.2,1,1,131,88,91,180,203,38,102,15.9,1,1.0,20,27,30,1
4,30,165,60,80.5,1.5,1.0,1,1,121,76,91,155,87,44,93,15.4,1,0.8,19,13,17,0


# Preprocessing & Engineering

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159256 entries, 0 to 159255
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   age                  159256 non-null  int64  
 1   height(cm)           159256 non-null  int64  
 2   weight(kg)           159256 non-null  int64  
 3   waist(cm)            159256 non-null  float64
 4   eyesight(left)       159256 non-null  float64
 5   eyesight(right)      159256 non-null  float64
 6   hearing(left)        159256 non-null  int64  
 7   hearing(right)       159256 non-null  int64  
 8   systolic             159256 non-null  int64  
 9   relaxation           159256 non-null  int64  
 10  fasting blood sugar  159256 non-null  int64  
 11  Cholesterol          159256 non-null  int64  
 12  triglyceride         159256 non-null  int64  
 13  HDL                  159256 non-null  int64  
 14  LDL                  159256 non-null  int64  
 15  hemoglobin       

In [8]:
def generate_unique_count_summary(df):
    unique_count_df = pd.DataFrame(
        {"name": df.columns, "nunique": df.nunique(), "dtype": df.dtypes}
    )
    unique_count_df["fraction"] = round((unique_count_df["nunique"] / len(df)).mul(100), 3)
    unique_count_df = unique_count_df.sort_values(by="fraction")
    unique_count_df = unique_count_df.reset_index(drop=True)
    return unique_count_df

In [9]:
unique_count_df = generate_unique_count_summary(X)

intial_cat_features = unique_count_df[unique_count_df['nunique'] <= 6]['name'].tolist()
num_features = [col for col in X.columns if col not in intial_cat_features]
intial_cat_features

['dental caries', 'hearing(right)', 'hearing(left)', 'Urine protein']

In [10]:
X.shape

(159256, 22)

In [11]:
def feature_engineering(df, numerical_features, cat_features):
    """
    Perform the specified feature engineering tasks on a DataFrame.
    
    Args:
    - df: DataFrame, input data with features.
    
    Returns:
    - df_new: DataFrame, data with new engineered features.
    """
    df_new = df.copy()
    
    # 1. Transformation of numerical features
    for feature in numerical_features:
        # Log transformation (add small value to avoid log(0))
        df_new[f"{feature}_log"] = np.log(df[feature] + 1e-10)
        
        # Square root transformation
        df_new[f"{feature}_sqrt"] = np.sqrt(df[feature])
        
        # Polynomial transformations
        df_new[f"{feature}_degree2"] = df[feature]**2
        df_new[f"{feature}_degree3"] = df[feature]**3
    
    # 2. Pairwise numerical feature operations
    for i in range(len(numerical_features)):
        for j in range(i, len(numerical_features)):
            if i != j:
                feat1 = numerical_features[i]
                feat2 = numerical_features[j]
                
                # Multiplication
                df_new[f"{feat1}_x_{feat2}"] = df[feat1] * df[feat2]
                
                # Addition
                df_new[f"{feat1}_plus_{feat2}"] = df[feat1] + df[feat2]
                
                # Subtraction (both ways)
                df_new[f"{feat1}_minus_{feat2}"] = df[feat1] - df[feat2]
                df_new[f"{feat2}_minus_{feat1}"] = df[feat2] - df[feat1]
                
                # Ratio (both ways)
                df_new[f"{feat1}_div_{feat2}"] = df[feat1] / (df[feat2] + 1e-10)
                df_new[f"{feat2}_div_{feat1}"] = df[feat2] / (df[feat1] + 1e-10)

                # Mean
                df_new[f"Mean_{feat1}_{feat2}"] = df[[feat1, feat2]].mean(axis=1)

                # Standard Deviation
                df_new[f"Std_{feat1}_{feat2}"] = df[[feat1, feat2]].std(axis=1)

                # Percentage Sum (both ways)
                df_new[f"percent_sum_{feat1}_{feat2}"] = (df[feat1] / (df[feat1] + df[feat2])) * 100
                df_new[f"percent_sum_{feat2}_{feat1}"] = (df[feat2] / (df[feat1] + df[feat2])) * 100

    # 3. Categorical feature operations
    for i in range(len(cat_features)):
        for j in range(len(cat_features)):
            if i != j:
                cat_feat1 = cat_features[i]
                cat_feat2 = cat_features[j]

                # # Add the categorical features
                # df_new[f"{cat_feat1}_plus_{cat_feat2}"] = df[cat_feat1] + df[cat_feat2]

                # # Cross Tabulation
                # cross_tab = pd.crosstab(df[cat_feat1], df[cat_feat2])
                # cross_tab.columns = [f"{cat_feat2}_{col}" for col in cross_tab.columns]
                # cross_tab = cross_tab.reset_index()

                # df_new = df_new.merge(cross_tab, how='left', on=cat_feat1)

                # Pair Counting
                df_new[f"Count_{cat_feat1}_{cat_feat2}"] = df.groupby([cat_feat1, cat_feat2])[cat_feat1].transform('count')

    # 4. Numerical-Categorical feature engineering
    for cat_col in cat_features:
        for num_col in num_features:
            df_new[f"mean_{num_col}_by_{cat_col}"] = df.groupby(cat_col)[num_col].transform('mean')

            df_new[f"interact_{num_col}_{cat_col}"] = df[cat_col] * df[num_col]

            df_new[f"ratio_to_group_mean_{num_col}_by_{cat_col}"] = df[num_col] / df.groupby(cat_col)[num_col].transform('mean')
    
    # 5. Domain Knowledge features
    df_new['BMI'] = df['weight(kg)'] / ((df['height(cm)'] * 0.01)**2)
    df['BMI'] = df['weight(kg)'] / ((df['height(cm)'] * 0.01)**2)

    df_new['Gtp_to_Liver_Function_Index'] = df['Gtp'] / (df['AST'] + df['ALT'])

    df_new['age_waist_adjusted'] = df['waist(cm)'] * (df['age'] / 50)

    df_new['blood_sugar_waist_adjusted'] = df['fasting blood sugar'] * (df['waist(cm)'] / 100)

    df_new['BMI_body_index'] = df['BMI'] / df['systolic']

    df_new['BMI_risk'] = np.select([df['BMI'] < 18.5, df['BMI'] < 25, df['BMI'] < 30, df['BMI'] < 35, df['BMI'] < 40], [0, 1, 2, 3, 4], 5)

    df_new['BP_risk'] = np.select([(df['systolic'] < 120) | (df['relaxation'] < 80), (df['systolic'] < 130) | (df['relaxation'] < 90), (df['systolic'] < 140) | (df['relaxation'] < 100), (df['systolic'] < 180) | (df['relaxation'] < 120)], [0, 1, 2, 3], 4)
    
    df_new['tot_chol_risk'] = np.where(df['Cholesterol'] >= 200, 1, 0)

    df_new['HDL_risk'] = np.select([df['HDL'] < 40, df['HDL'] <= 60], [0, 1], 2)

    df_new['LDL_risk'] = np.select([df['LDL'] < 100, df['LDL'] < 130, df['LDL'] < 160, df['LDL'] < 190], [0, 1, 2, 3], 4)

    df_new['triglyceride_risk'] = np.select([df['triglyceride'] < 150, df['triglyceride'] < 200, df['triglyceride'] < 500], [0, 1, 2], 3)

    df_new['hemoglobin_risk'] = np.where(df['hemoglobin'].between(13.8, 17.2, inclusive = "left"), 0, 1)

    df_new['Gtp_risk'] = np.where(df['Gtp'].between(5, 40, inclusive = "left"), 1, 0)

    df_new['AST_risk'] = np.where(df['AST'].between(8, 45, inclusive = "left"), 1, 0)

    df_new['ALT_risk'] = np.where(df['ALT'].between(8, 45, inclusive = "left"), 1, 0)

    return df_new

In [12]:
X = feature_engineering(X, num_features, intial_cat_features)
X.head()

age  height(cm)  weight(kg)  waist(cm)  eyesight(left)  eyesight(right)  \
0   55         165          60       81.0             0.5              0.6   
1   70         165          65       89.0             0.6              0.7   
2   20         170          75       81.0             0.4              0.5   
3   35         180          95      105.0             1.5              1.2   
4   30         165          60       80.5             1.5              1.0   

   hearing(left)  hearing(right)  systolic  relaxation  fasting blood sugar  \
0              1               1       135          87                   94   
1              2               2       146          83                  147   
2              1               1       118          75                   79   
3              1               1       131          88                   91   
4              1               1       121          76                   91   

   Cholesterol  triglyceride  HDL  LDL  hemoglobin  Urine protein  \
0          172           300   40   75        16.5              1   
1          194            55   57  126        16.2              1   
2          178           197   45   93        17.4              1   
3          180           203   38  102        15.9              1   
4          155            87   44   93        15.4              1   

   serum creatinine  AST  ALT  Gtp  dental caries   age_log  age_sqrt  \
0               1.0   22   25   27              0  4.007333  7.416198   
1               1.1   27   23   37              1  4.248495  8.366600   
2               0.8   27   31   53              0  2.995732  4.472136   
3               1.0   20   27   30              1  3.555348  5.916080   
4               0.8   19   13   17              0  3.401197  5.477226   

   age_degree2  age_degree3  height(cm)_log  height(cm)_sqrt  \
0         3025       166375        5.105945        12.845233   
1         4900       343000        5.105945        12.845233   
2          400         8000        5.135798        13.038405   
3         1225        42875        5.192957        13.416408   
4          900        27000        5.105945        12.845233   

   height(cm)_degree2  height(cm)_degree3  weight(kg)_log  weight(kg)_sqrt  \
0               27225             4492125        4.094345         7.745967   
1               27225             4492125        4.174387         8.062258   
2               28900             4913000        4.317488         8.660254   
3               32400             5832000        4.553877         9.746794   
4               27225             4492125        4.094345         7.745967   

   weight(kg)_degree2  weight(kg)_degree3  waist(cm)_log  waist(cm)_sqrt  \
0                3600              216000       4.394449        9.000000   
1                4225              274625       4.488636        9.433981   
2                5625              421875       4.394449        9.000000   
3                9025              857375       4.653960       10.246951   
4                3600              216000       4.388257        8.972179   

   waist(cm)_degree2  waist(cm)_degree3  eyesight(left)_log  \
0            6561.00         531441.000           -0.693147   
1            7921.00         704969.000           -0.510826   
2            6561.00         531441.000           -0.916291   
3           11025.00        1157625.000            0.405465   
4            6480.25         521660.125            0.405465   

   eyesight(left)_sqrt  eyesight(left)_degree2  eyesight(left)_degree3  \
0             0.707107                    0.25                   0.125   
1             0.774597                    0.36                   0.216   
2             0.632456                    0.16                   0.064   
3             1.224745                    2.25                   3.375   
4             1.224745                    2.25                   3.375   

   eyesight(right)_log  eyesight(right)_sqrt  eyesight(right)_degree2 

In [13]:
X.shape

(159256, 1867)

In [14]:
unique_count_df = generate_unique_count_summary(X)

cat_features = unique_count_df[unique_count_df['nunique'] <= 6]['name'].tolist()
num_features = [col for col in X.columns if col not in cat_features]

In [15]:
standard_scaler = StandardScaler().fit_transform(X[num_features])
train_scaled = pd.DataFrame(standard_scaler, columns=num_features)
X = pd.concat([X[cat_features], train_scaled], axis=1)
X.head()

ALT_risk  mean_eyesight(right)_by_hearing(right)  \
0         1                                1.005745   
1         1                                0.802681   
2         1                                1.005745   
3         1                                1.005745   
4         1                                1.005745   

   mean_systolic_by_hearing(right)  mean_relaxation_by_hearing(right)  \
0                       122.421267                          76.895310   
1                       125.938606                          75.988472   
2                       122.421267                          76.895310   
3                       122.421267                          76.895310   
4                       122.421267                          76.895310   

   mean_fasting blood sugar_by_hearing(right)  \
0                                   98.293244   
1                                  100.825469   
2                                   98.293244   
3                                   98.293244   
4                                   98.293244   

   mean_Cholesterol_by_hearing(right)  mean_triglyceride_by_hearing(right)  \
0                          195.924913                            127.96807   
1                          190.427882                            112.93807   
2                          195.924913                            127.96807   
3                          195.924913                            127.96807   
4                          195.924913                            127.96807   

   mean_HDL_by_hearing(right)  mean_LDL_by_hearing(right)  \
0                   55.859843                  114.662031   
1                   55.554155                  112.341555   
2                   55.859843                  114.662031   
3                   55.859843                  114.662031   
4                   55.859843                  114.662031   

   mean_hemoglobin_by_hearing(right)  mean_serum creatinine_by_hearing(right)  \
0                          14.807970                                 0.893022   
1                          14.338123                                 0.882038   
2                          14.807970                                 0.893022   
3                          14.807970                                 0.893022   
4                          14.807970                                 0.893022   

   mean_AST_by_hearing(right)  mean_ALT_by_hearing(right)  \
0                   25.518659                   26.651139   
1                   25.441555                   22.345576   
2                   25.518659                   26.651139   
3                   25.518659                   26.651139   
4                   25.518659                   26.651139   

   mean_Gtp_by_hearing(right)  mean_age_by_hearing(left)  \
0                   36.299538                  43.911508   
1                   32.732976                  60.392614   
2                   36.299538                  43.911508   
3                   36.299538                  43.911508   
4                   36.299538                  43.911508   

   mean_weight(kg)_by_hearing(left)  mean_waist(cm)_by_hearing(left)  \
0                         67.281270                        82.995308   
1                         61.541383                        83.274018   
2                         67.281270                        82.995308   
3                         67.281270                        82.995308   
4                         67.281270                        82.995308   

   mean_eyesight(left)_by_hearing(left)  \
0                              1.010070   
1                              0.831875   
2                              1.010070   
3                              1.010070   
4                              1.010070   

   mean_eyesight(right)_by_hearing(left)  mean_systolic_by_hearing(left)  \
0                               1.005470                      122.420084   
1                               0.818544   

# Remove correlated features

In [16]:
# %%time

# correlation_matrix = X.corr().abs()
# print('Correlation complete!')

In [17]:

# # Select upper triangle of correlation matrix
# upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# # Find index of feature columns with correlation greater than a threshold (e.g., 0.8 in this case)
# to_drop = [column for column in upper.columns if any(upper[column] >= 0.8)]

# # Drop features
# df_reduced = X.drop(to_drop, axis=1)

# print(f"Dropped {len(to_drop)} highly correlated features.")

In [18]:
# list(df_reduced.columns)

In [19]:
# len(df_reduced.columns)

In [20]:
uncorrelated_features = ['ALT_risk', 'mean_eyesight(right)_by_hearing(right)', 'mean_age_by_hearing(left)', 'mean_age_by_dental caries', 'Gtp_risk', 'hemoglobin_risk', 'tot_chol_risk', 'AST_risk', 'HDL_risk', 'LDL_risk', 'BP_risk', 'triglyceride_risk', 'mean_AST_by_Urine protein', 'BMI_risk', 'mean_hemoglobin_by_Urine protein', 'mean_height(cm)_by_Urine protein', 'mean_eyesight(left)_by_Urine protein', 'age', 'height(cm)', 'weight(kg)', 'eyesight(left)', 'eyesight(right)', 'systolic', 'fasting blood sugar', 'hemoglobin', 'serum creatinine', 'AST', 'ALT', 'Gtp', 'eyesight(left)_degree2', 'eyesight(right)_degree2', 'LDL_degree2', 'serum creatinine_degree3', 'AST_degree2', 'ALT_degree2', 'Gtp_degree2', 'age_x_eyesight(left)', 'eyesight(left)_div_age', 'age_x_eyesight(right)', 'eyesight(right)_div_age', 'age_minus_systolic', 'age_minus_fasting blood sugar', 'age_x_HDL', 'age_minus_HDL', 'Std_age_HDL', 'age_div_LDL', 'age_x_serum creatinine', 'age_x_AST', 'age_minus_AST', 'Std_age_ALT', 'Std_age_Gtp', 'height(cm)_minus_waist(cm)', 'Std_height(cm)_triglyceride', 'height(cm)_div_LDL', 'height(cm)_minus_AST', 'Std_height(cm)_ALT', 'Std_height(cm)_Gtp', 'weight(kg)_minus_waist(cm)', 'Std_weight(kg)_relaxation', 'weight(kg)_x_Cholesterol', 'Std_weight(kg)_HDL', 'weight(kg)_div_serum creatinine', 'weight(kg)_minus_AST', 'Std_weight(kg)_ALT', 'Std_weight(kg)_Gtp', 'Std_waist(cm)_relaxation', 'waist(cm)_div_hemoglobin', 'eyesight(left)_x_eyesight(right)', 'eyesight(left)_minus_eyesight(right)', 'eyesight(left)_div_eyesight(right)', 'eyesight(right)_div_eyesight(left)', 'Std_eyesight(left)_eyesight(right)', 'percent_sum_eyesight(left)_eyesight(right)', 'eyesight(left)_x_triglyceride', 'eyesight(left)_div_triglyceride', 'triglyceride_div_eyesight(left)', 'eyesight(left)_div_LDL', 'Std_eyesight(left)_serum creatinine', 'eyesight(left)_x_AST', 'eyesight(left)_div_ALT', 'eyesight(left)_div_Gtp', 'Gtp_div_eyesight(left)', 'eyesight(right)_x_triglyceride', 'triglyceride_div_eyesight(right)', 'eyesight(right)_div_LDL', 'Std_eyesight(right)_serum creatinine', 'eyesight(right)_x_AST', 'eyesight(right)_div_AST', 'eyesight(right)_div_ALT', 'ALT_div_eyesight(right)', 'Gtp_div_eyesight(right)', 'systolic_minus_relaxation', 'systolic_minus_fasting blood sugar', 'Std_systolic_fasting blood sugar', 'Std_systolic_triglyceride', 'systolic_minus_HDL', 'Std_systolic_LDL', 'systolic_div_hemoglobin', 'systolic_minus_AST', 'Std_systolic_ALT', 'Std_systolic_Gtp', 'fasting blood sugar_x_Cholesterol', 'Std_fasting blood sugar_LDL', 'fasting blood sugar_x_serum creatinine', 'fasting blood sugar_div_serum creatinine', 'fasting blood sugar_minus_ALT', 'Std_fasting blood sugar_ALT', 'Std_fasting blood sugar_Gtp', 'Std_Cholesterol_triglyceride', 'Cholesterol_minus_LDL', 'LDL_div_Cholesterol', 'Std_Cholesterol_LDL', 'Cholesterol_minus_Gtp', 'triglyceride_x_AST', 'AST_div_triglyceride', 'triglyceride_div_ALT', 'ALT_div_triglyceride', 'triglyceride_div_Gtp', 'Gtp_div_triglyceride', 'HDL_x_serum creatinine', 'Std_HDL_Gtp', 'AST_div_LDL', 'ALT_div_LDL', 'Gtp_div_LDL', 'Std_LDL_Gtp', 'AST_div_ALT', 'Std_AST_ALT', 'AST_x_Gtp', 'AST_div_Gtp', 'interact_Gtp_dental caries', 'interact_weight(kg)_hearing(right)', 'interact_fasting blood sugar_hearing(right)', 'interact_Cholesterol_hearing(right)', 'interact_fasting blood sugar_hearing(left)', 'interact_Cholesterol_hearing(left)', 'interact_age_Urine protein', 'interact_eyesight(left)_Urine protein', 'interact_eyesight(right)_Urine protein', 'interact_HDL_Urine protein', 'interact_AST_Urine protein']
# uncorrelated_features = ['ALT_risk', 'mean_eyesight(right)_by_hearing(right)', 'mean_age_by_hearing(left)', 'mean_age_by_dental caries', 'Gtp_risk', 'hemoglobin_risk', 'tot_chol_risk', 'AST_risk', 'HDL_risk', 'LDL_risk', 'BP_risk', 'Count_hearing(left)_hearing(right)', 'triglyceride_risk', 'mean_AST_by_Urine protein', 'BMI_risk', 'mean_hemoglobin_by_Urine protein']
len(uncorrelated_features)

140

['ALT_risk', 'mean_eyesight(right)_by_hearing(right)', 'mean_age_by_hearing(left)', 'mean_age_by_dental caries', 'Gtp_risk', 'hemoglobin_risk', 'tot_chol_risk', 'AST_risk', 'HDL_risk', 'LDL_risk', 'BP_risk', 'triglyceride_risk', 'mean_AST_by_Urine protein', 'BMI_risk', 'mean_hemoglobin_by_Urine protein', 'mean_height(cm)_by_Urine protein', 'mean_eyesight(left)_by_Urine protein', 'age', 'height(cm)', 'weight(kg)', 'eyesight(left)', 'eyesight(right)', 'systolic', 'fasting blood sugar', 'hemoglobin', 'serum creatinine', 'AST', 'ALT', 'Gtp', 'eyesight(left)_degree2', 'eyesight(right)_degree2', 'LDL_degree2', 'serum creatinine_degree3', 'AST_degree2', 'ALT_degree2', 'Gtp_degree2', 'age_x_eyesight(left)', 'eyesight(left)_div_age', 'age_x_eyesight(right)', 'eyesight(right)_div_age', 'age_minus_systolic', 'age_minus_fasting blood sugar', 'age_x_HDL', 'age_minus_HDL', 'Std_age_HDL', 'age_div_LDL', 'age_x_serum creatinine', 'age_x_AST', 'age_minus_AST', 'Std_age_ALT', 'Std_age_Gtp', 'height(cm)_minus_waist(cm)', 'Std_height(cm)_triglyceride', 'height(cm)_div_LDL', 'height(cm)_minus_AST', 'Std_height(cm)_ALT', 'Std_height(cm)_Gtp', 'weight(kg)_minus_waist(cm)', 'Std_weight(kg)_relaxation', 'weight(kg)_x_Cholesterol', 'Std_weight(kg)_HDL', 'weight(kg)_div_serum creatinine', 'weight(kg)_minus_AST', 'Std_weight(kg)_ALT', 'Std_weight(kg)_Gtp', 'Std_waist(cm)_relaxation', 'waist(cm)_div_hemoglobin', 'eyesight(left)_x_eyesight(right)', 'eyesight(left)_minus_eyesight(right)', 'eyesight(left)_div_eyesight(right)', 'eyesight(right)_div_eyesight(left)', 'Std_eyesight(left)_eyesight(right)', 'percent_sum_eyesight(left)_eyesight(right)', 'eyesight(left)_x_triglyceride', 'eyesight(left)_div_triglyceride', 'triglyceride_div_eyesight(left)', 'eyesight(left)_div_LDL', 'Std_eyesight(left)_serum creatinine', 'eyesight(left)_x_AST', 'eyesight(left)_div_ALT', 'eyesight(left)_div_Gtp', 'Gtp_div_eyesight(left)', 'eyesight(right)_x_triglyceride', 'triglyceride_div_eyesight(right)', 'eyesight(right)_div_LDL', 'Std_eyesight(right)_serum creatinine', 'eyesight(right)_x_AST', 'eyesight(right)_div_AST', 'eyesight(right)_div_ALT', 'ALT_div_eyesight(right)', 'Gtp_div_eyesight(right)', 'systolic_minus_relaxation', 'systolic_minus_fasting blood sugar', 'Std_systolic_fasting blood sugar', 'Std_systolic_triglyceride', 'systolic_minus_HDL', 'Std_systolic_LDL', 'systolic_div_hemoglobin', 'systolic_minus_AST', 'Std_systolic_ALT', 'Std_systolic_Gtp', 'fasting blood sugar_x_Cholesterol', 'Std_fasting blood sugar_LDL', 'fasting blood sugar_x_serum creatinine', 'fasting blood sugar_div_serum creatinine', 'fasting blood sugar_minus_ALT', 'Std_fasting blood sugar_ALT', 'Std_fasting blood sugar_Gtp', 'Std_Cholesterol_triglyceride', 'Cholesterol_minus_LDL', 'LDL_div_Cholesterol', 'Std_Cholesterol_LDL', 'Cholesterol_minus_Gtp', 'triglyceride_x_AST', 'AST_div_triglyceride', 'triglyceride_div_ALT', 'ALT_div_triglyceride', 'triglyceride_div_Gtp', 'Gtp_div_triglyceride', 'HDL_x_serum creatinine', 'Std_HDL_Gtp', 'AST_div_LDL', 'ALT_div_LDL', 'Gtp_div_LDL', 'Std_LDL_Gtp', 'AST_div_ALT', 'Std_AST_ALT', 'AST_x_Gtp', 'AST_div_Gtp', 'interact_Gtp_dental caries', 'interact_weight(kg)_hearing(right)', 'interact_fasting blood sugar_hearing(right)', 'interact_Cholesterol_hearing(right)', 'interact_fasting blood sugar_hearing(left)', 'interact_Cholesterol_hearing(left)', 'interact_age_Urine protein', 'interact_eyesight(left)_Urine protein', 'interact_eyesight(right)_Urine protein', 'interact_HDL_Urine protein', 'interact_AST_Urine protein']

['ALT_risk', 'mean_eyesight(right)_by_hearing(right)', 'mean_age_by_hearing(left)', 'mean_age_by_dental caries', 'Gtp_risk', 'hemoglobin_risk', 'tot_chol_risk', 'AST_risk', 'HDL_risk', 'LDL_risk', 'BP_risk', 'Count_hearing(left)_hearing(right)', 'triglyceride_risk', 'mean_AST_by_Urine protein', 'BMI_risk', 'mean_hemoglobin_by_Urine protein', 'mean_height(cm)_by_Urine protein', 'mean_eyesight(left)_by_Urine protein', 'mean_serum creatinine_by_Urine protein', 'age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)', 'eyesight(right)', 'systolic', 'relaxation', 'fasting blood sugar', 'Cholesterol', 'triglyceride', 'HDL', 'LDL', 'hemoglobin', 'serum creatinine', 'AST', 'ALT', 'Gtp', 'eyesight(left)_log', 'eyesight(left)_degree2', 'eyesight(right)_log', 'eyesight(right)_degree2', 'LDL_degree2', 'serum creatinine_degree2', 'serum creatinine_degree3', 'AST_degree2', 'ALT_log', 'ALT_degree2', 'Gtp_log', 'Gtp_degree2', 'age_plus_height(cm)', 'age_x_weight(kg)', 'age_minus_weight(kg)', 'age_x_eyesight(left)', 'age_div_eyesight(left)', 'eyesight(left)_div_age', 'age_x_eyesight(right)', 'age_div_eyesight(right)', 'eyesight(right)_div_age', 'age_minus_systolic', 'age_minus_relaxation', 'age_x_fasting blood sugar', 'age_minus_fasting blood sugar', 'age_div_fasting blood sugar', 'age_x_Cholesterol', 'age_div_Cholesterol', 'age_x_triglyceride', 'age_div_triglyceride', 'age_x_HDL', 'age_minus_HDL', 'Std_age_HDL', 'age_div_LDL', 'LDL_div_age', 'age_x_serum creatinine', 'age_div_serum creatinine', 'serum creatinine_div_age', 'age_x_AST', 'age_minus_AST', 'age_div_AST', 'Std_age_AST', 'age_x_ALT', 'age_minus_ALT', 'age_div_ALT', 'Std_age_ALT', 'percent_sum_age_ALT', 'age_div_Gtp', 'Std_age_Gtp', 'height(cm)_minus_weight(kg)', 'height(cm)_minus_waist(cm)', 'height(cm)_div_waist(cm)', 'height(cm)_x_systolic', 'height(cm)_minus_systolic', 'height(cm)_minus_relaxation', 'height(cm)_minus_fasting blood sugar', 'Std_height(cm)_Cholesterol', 'Std_height(cm)_triglyceride', 'height(cm)_minus_HDL', 'height(cm)_div_LDL', 'Std_height(cm)_LDL', 'height(cm)_div_hemoglobin', 'height(cm)_plus_AST', 'height(cm)_minus_AST', 'Std_height(cm)_AST', 'height(cm)_minus_ALT', 'Std_height(cm)_ALT', 'Std_height(cm)_Gtp', 'weight(kg)_minus_waist(cm)', 'weight(kg)_minus_systolic', 'weight(kg)_minus_relaxation', 'Std_weight(kg)_relaxation', 'weight(kg)_x_fasting blood sugar', 'weight(kg)_minus_fasting blood sugar', 'weight(kg)_x_Cholesterol', 'weight(kg)_div_Cholesterol', 'weight(kg)_x_HDL', 'Std_weight(kg)_HDL', 'weight(kg)_x_LDL', 'weight(kg)_div_hemoglobin', 'weight(kg)_x_serum creatinine', 'weight(kg)_div_serum creatinine', 'serum creatinine_div_weight(kg)', 'weight(kg)_minus_AST', 'weight(kg)_div_AST', 'Std_weight(kg)_AST', 'weight(kg)_minus_ALT', 'Std_weight(kg)_ALT', 'Std_weight(kg)_Gtp', 'waist(cm)_minus_systolic', 'waist(cm)_minus_relaxation', 'Std_waist(cm)_relaxation', 'waist(cm)_x_HDL', 'waist(cm)_div_hemoglobin', 'waist(cm)_div_serum creatinine', 'serum creatinine_div_waist(cm)', 'waist(cm)_minus_AST', 'Std_waist(cm)_AST', 'Std_waist(cm)_ALT', 'Std_waist(cm)_Gtp', 'eyesight(left)_x_eyesight(right)', 'eyesight(left)_plus_eyesight(right)', 'eyesight(left)_minus_eyesight(right)', 'eyesight(left)_div_eyesight(right)', 'eyesight(right)_div_eyesight(left)', 'Std_eyesight(left)_eyesight(right)', 'percent_sum_eyesight(left)_eyesight(right)', 'eyesight(left)_x_triglyceride', 'eyesight(left)_div_triglyceride', 'triglyceride_div_eyesight(left)', 'eyesight(left)_x_HDL', 'eyesight(left)_div_HDL', 'eyesight(left)_div_LDL', 'Std_eyesight(left)_serum creatinine', 'percent_sum_eyesight(left)_serum creatinine', 'eyesight(left)_x_AST', 'eyesight(left)_div_AST', 'AST_div_eyesight(left)', 'eyesight(left)_x_ALT', 'eyesight(left)_div_ALT', 'ALT_div_eyesight(left)', 'eyesight(left)_x_Gtp', 'eyesight(left)_div_Gtp', 'Gtp_div_eyesight(left)', 'eyesight(right)_x_triglyceride', 'eyesight(right)_div_triglyceride', 'triglyceride_div_eyesight(right)', 'eyesight(right)_x_HDL', 'eyesight(right)_div_HDL', 'eyesight(right)_div_LDL', 'Std_eyesight(right)_serum creatinine', 'percent_sum_eyesight(right)_serum creatinine', 'eyesight(right)_x_AST', 'eyesight(right)_div_AST', 'AST_div_eyesight(right)', 'eyesight(right)_x_ALT', 'eyesight(right)_div_ALT', 'ALT_div_eyesight(right)', 'eyesight(right)_x_Gtp', 'eyesight(right)_div_Gtp', 'Gtp_div_eyesight(right)', 'systolic_minus_relaxation', 'systolic_div_relaxation', 'systolic_x_fasting blood sugar', 'systolic_minus_fasting blood sugar', 'Std_systolic_fasting blood sugar', 'systolic_x_Cholesterol', 'Std_systolic_triglyceride', 'systolic_minus_HDL', 'Std_systolic_LDL', 'systolic_x_hemoglobin', 'systolic_div_hemoglobin', 'systolic_x_serum creatinine', 'systolic_div_serum creatinine', 'systolic_plus_AST', 'systolic_minus_AST', 'systolic_minus_ALT', 'Std_systolic_ALT', 'Std_systolic_Gtp', 'Std_relaxation_HDL', 'relaxation_x_hemoglobin', 'relaxation_div_hemoglobin', 'relaxation_minus_AST', 'Std_relaxation_AST', 'Std_relaxation_ALT', 'Std_relaxation_Gtp', 'fasting blood sugar_x_Cholesterol', 'fasting blood sugar_minus_Cholesterol', 'fasting blood sugar_x_HDL', 'fasting blood sugar_minus_HDL', 'fasting blood sugar_x_LDL', 'fasting blood sugar_minus_LDL', 'Std_fasting blood sugar_LDL', 'fasting blood sugar_div_hemoglobin', 'fasting blood sugar_x_serum creatinine', 'fasting blood sugar_div_serum creatinine', 'serum creatinine_div_fasting blood sugar', 'fasting blood sugar_plus_AST', 'fasting blood sugar_minus_AST', 'fasting blood sugar_div_AST', 'fasting blood sugar_plus_ALT', 'fasting blood sugar_minus_ALT', 'Std_fasting blood sugar_ALT', 'fasting blood sugar_minus_Gtp', 'Std_fasting blood sugar_Gtp', 'Std_Cholesterol_triglyceride', 'Cholesterol_x_HDL', 'Cholesterol_div_HDL', 'Cholesterol_minus_LDL', 'LDL_div_Cholesterol', 'Std_Cholesterol_LDL', 'percent_sum_Cholesterol_LDL', 'Cholesterol_x_hemoglobin', 'Cholesterol_div_hemoglobin', 'Cholesterol_x_serum creatinine', 'Cholesterol_div_serum creatinine', 'Cholesterol_div_AST', 'Cholesterol_plus_Gtp', 'Cholesterol_minus_Gtp', 'Std_Cholesterol_Gtp', 'triglyceride_x_HDL', 'triglyceride_div_LDL', 'Std_triglyceride_LDL', 'triglyceride_x_AST', 'triglyceride_div_AST', 'AST_div_triglyceride', 'triglyceride_x_ALT', 'triglyceride_div_ALT', 'ALT_div_triglyceride', 'percent_sum_triglyceride_ALT', 'triglyceride_x_Gtp', 'triglyceride_minus_Gtp', 'triglyceride_div_Gtp', 'Gtp_div_triglyceride', 'percent_sum_triglyceride_Gtp', 'HDL_x_LDL', 'HDL_div_LDL', 'HDL_x_serum creatinine', 'HDL_div_serum creatinine', 'serum creatinine_div_HDL', 'HDL_x_AST', 'HDL_plus_AST', 'HDL_minus_AST', 'HDL_div_AST', 'AST_div_HDL', 'Std_HDL_AST', 'HDL_minus_ALT', 'HDL_div_ALT', 'Std_HDL_ALT', 'Std_HDL_Gtp', 'LDL_x_serum creatinine', 'LDL_div_serum creatinine', 'LDL_div_AST', 'AST_div_LDL', 'ALT_div_LDL', 'Gtp_div_LDL', 'Std_LDL_Gtp', 'serum creatinine_div_hemoglobin', 'serum creatinine_div_AST', 'AST_div_serum creatinine', 'AST_x_ALT', 'AST_minus_ALT', 'AST_div_ALT', 'Std_AST_ALT', 'percent_sum_AST_ALT', 'AST_x_Gtp', 'AST_div_Gtp', 'ALT_x_Gtp', 'ALT_minus_Gtp', 'Gtp_div_ALT', 'Std_ALT_Gtp', 'percent_sum_ALT_Gtp', 'Count_dental caries_Urine protein', 'Count_hearing(right)_Urine protein', 'Count_hearing(left)_Urine protein', 'interact_triglyceride_dental caries', 'interact_Gtp_dental caries', 'interact_age_hearing(right)', 'interact_weight(kg)_hearing(right)', 'interact_waist(cm)_hearing(right)', 'interact_systolic_hearing(right)', 'interact_fasting blood sugar_hearing(right)', 'interact_Cholesterol_hearing(right)', 'interact_HDL_hearing(right)', 'interact_LDL_hearing(right)', 'interact_hemoglobin_hearing(right)', 'interact_serum creatinine_hearing(right)', 'interact_age_hearing(left)', 'interact_weight(kg)_hearing(left)', 'interact_waist(cm)_hearing(left)', 'interact_systolic_hearing(left)', 'interact_fasting blood sugar_hearing(left)', 'interact_Cholesterol_hearing(left)', 'interact_HDL_hearing(left)', 'interact_LDL_hearing(left)', 'interact_hemoglobin_hearing(left)', 'interact_serum creatinine_hearing(left)', 'interact_age_Urine protein', 'interact_weight(kg)_Urine protein', 'interact_eyesight(left)_Urine protein', 'interact_eyesight(right)_Urine protein', 'interact_fasting blood sugar_Urine protein', 'interact_triglyceride_Urine protein', 'interact_HDL_Urine protein', 'interact_serum creatinine_Urine protein', 'interact_AST_Urine protein', 'interact_ALT_Urine protein', 'interact_Gtp_Urine protein', 'BMI_body_index']

# RFECV feature selection

In [21]:
def apply_rfecv(classifiers, X, y, feature_set):
    selected_features_dict = {}
    execution_times = {}

    for clf_name, clf in classifiers.items():
        start_time = time.time()
        print(f"Processing {clf_name}")

        X_train = X[feature_set]

        # Custom importance getter for HistGradientBoostingClassifier
        if clf_name in ['HistGradientBoosting', 'GaussianNB']:
            importance_getter = lambda est: permutation_importance(est, X_train, y, n_repeats=5, random_state=5).importances_mean
        else:
            importance_getter = 'auto'

        selector = RFECV(clf, cv=10, step=1, scoring='roc_auc', verbose=2, importance_getter=importance_getter)
        selector = selector.fit(X_train, y)

        selected_features = list(X_train.columns[selector.support_])
        selected_features_dict[clf_name] = selected_features
        print(f"Selected features for {clf_name}: {selected_features}")

        end_time = time.time()
        execution_time = end_time - start_time
        execution_times[clf_name] = execution_time
        print(f"Execution time for {clf_name} is {execution_time:.2f}")
        print()
        # # Optional: Visualize the ranking for the current classifier
        # plt.figure()
        # plt.bar(range(X_train.shape[1]), selector.ranking_)
        # plt.xticks(range(X_train.shape[1]), X_train.columns, rotation=90)
        # plt.ylabel('Feature Rank')
        # plt.title(f'Feature Ranking by RFECV - {clf_name}')
        # plt.show()

    return selected_features_dict, execution_times

In [22]:
classifiers = {
    # 'CatBoost': CatBoostClassifier(random_state=5, verbose=False, early_stopping_rounds=100),
    # 'GradientBoostingClassifier': GradientBoostingClassifier(random_state=5),
    # 'XGBoost': XGBClassifier(seed=5),
    # 'HistGradientBoosting': HistGradientBoostingClassifier(random_state=5),
    # 'AdaBoostClassifier': AdaBoostClassifier(random_state=5),
    # 'LGBM': LGBMClassifier(n_jobs=-1, random_state=5),
    'LogisticRegression': LogisticRegression(random_state=5, n_jobs=-1),
    # 'GaussianNB': GaussianNB(),
    'RidgeClassifier': RidgeClassifier(random_state=5),
    'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis(),
}

selected_features_dict, execution_times = apply_rfecv(classifiers, X, y, uncorrelated_features)

Processing LogisticRegression
Fitting estimator with 140 features.
Fitting estimator with 139 features.
Fitting estimator with 138 features.
Fitting estimator with 137 features.
Fitting estimator with 136 features.
Fitting estimator with 135 features.
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.
Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
Fitting estimator with 127 features.
Fitting estimator with 126 features.
Fitting estimator with 125 features.
Fitting estimator with 124 features.
Fitting estimator with 123 features.
Fitting estimator with 122 features.
Fitting estimator with 121 features.
Fitting estimator with 120 features.
Fitting estimator with 119 features.
Fitting estimator with 118 features.
Fitting estimator with 117 features.
Fitting estimator with 116 features.
Fitting estimator with 115 features.
Fitting 

In [23]:
print("Selected features for each classifier:")
pprint(selected_features_dict)

Selected features for each classifier:
{'LinearDiscriminantAnalysis': ['ALT_risk',
                                'mean_eyesight(right)_by_hearing(right)',
                                'mean_age_by_hearing(left)',
                                'mean_age_by_dental caries',
                                'Gtp_risk',
                                'hemoglobin_risk',
                                'tot_chol_risk',
                                'AST_risk',
                                'HDL_risk',
                                'LDL_risk',
                                'BP_risk',
                                'triglyceride_risk',
                                'mean_AST_by_Urine protein',
                                'BMI_risk',
                                'mean_hemoglobin_by_Urine protein',
                                'mean_height(cm)_by_Urine protein',
                                'mean_eyesight(left)_by_Urine protein',
                                'age

In [24]:
pprint(execution_times)

{'LinearDiscriminantAnalysis': 3176.8529155254364,
 'LogisticRegression': 5402.632709503174,
 'RidgeClassifier': 420.7602174282074}


In [25]:
hist_feats = []
lgbm_feats = ['BP_risk', 'age', 'height(cm)', 'weight(kg)', 'systolic', 'fasting blood sugar', 'hemoglobin', 'serum creatinine', 'AST', 'ALT', 'Gtp', 'LDL_degree2', 'age_x_eyesight(left)', 'age_minus_systolic', 'age_minus_fasting blood sugar', 'age_x_HDL', 'age_minus_HDL', 'age_div_LDL', 'age_x_serum creatinine', 'age_x_AST', 'age_minus_AST', 'height(cm)_minus_waist(cm)', 'Std_height(cm)_triglyceride', 'height(cm)_div_LDL', 'height(cm)_minus_AST', 'Std_height(cm)_ALT', 'weight(kg)_minus_waist(cm)', 'weight(kg)_x_Cholesterol', 'weight(kg)_div_serum creatinine', 'weight(kg)_minus_AST', 'Std_weight(kg)_Gtp', 'waist(cm)_div_hemoglobin', 'eyesight(left)_x_eyesight(right)', 'eyesight(left)_div_eyesight(right)', 'eyesight(left)_x_triglyceride', 'eyesight(left)_div_triglyceride', 'eyesight(left)_div_Gtp', 'eyesight(right)_x_triglyceride', 'triglyceride_div_eyesight(right)', 'ALT_div_eyesight(right)', 'Gtp_div_eyesight(right)', 'systolic_minus_relaxation', 'systolic_minus_fasting blood sugar', 'Std_systolic_triglyceride', 'systolic_div_hemoglobin', 'Std_systolic_Gtp', 'fasting blood sugar_x_serum creatinine', 'fasting blood sugar_div_serum creatinine', 'fasting blood sugar_minus_ALT', 'Std_Cholesterol_triglyceride', 'LDL_div_Cholesterol', 'Cholesterol_minus_Gtp', 'triglyceride_x_AST', 'AST_div_triglyceride', 'triglyceride_div_ALT', 'ALT_div_triglyceride', 'HDL_x_serum creatinine', 'Std_HDL_Gtp', 'Gtp_div_LDL', 'Std_LDL_Gtp', 'AST_div_ALT', 'AST_div_Gtp', 'interact_Gtp_dental caries', 'interact_fasting blood sugar_hearing(right)', 'interact_Cholesterol_hearing(right)', 'interact_Cholesterol_hearing(left)', 'interact_age_Urine protein', 'interact_HDL_Urine protein']
xgb_feats = []
catboost_feats = []
ada_feats = []
gradientboost_feats = []
logreg_feats = ['mean_age_by_dental caries',    'Gtp_risk',    'hemoglobin_risk',    'AST_risk',    'HDL_risk',    'LDL_risk',    'triglyceride_risk',    'mean_AST_by_Urine protein',    'height(cm)',    'eyesight(left)',    'eyesight(right)',    'fasting blood sugar',    'hemoglobin',    'serum creatinine',    'AST',    'Gtp',    'eyesight(left)_degree2',    'eyesight(right)_degree2',    'serum creatinine_degree3',    'eyesight(left)_div_age',    'eyesight(right)_div_age',    'age_minus_fasting blood sugar',    'age_x_HDL',    'Std_age_HDL',    'age_x_serum creatinine',    'age_x_AST',    'age_minus_AST',    'Std_age_Gtp',    'height(cm)_minus_waist(cm)',    'Std_height(cm)_triglyceride',    'height(cm)_div_LDL',    'height(cm)_minus_AST',    'Std_height(cm)_ALT',    'weight(kg)_minus_waist(cm)',    'weight(kg)_x_Cholesterol',    'weight(kg)_div_serum creatinine',    'Std_weight(kg)_Gtp',    'waist(cm)_div_hemoglobin',    'eyesight(left)_x_triglyceride',    'triglyceride_div_eyesight(left)',    'eyesight(left)_div_Gtp',    'eyesight(right)_x_triglyceride',    'eyesight(right)_div_AST',    'Std_systolic_triglyceride',    'systolic_minus_HDL',    'Std_systolic_LDL',    'systolic_div_hemoglobin',    'fasting blood sugar_x_Cholesterol',    'fasting blood sugar_x_serum creatinine',    'fasting blood sugar_div_serum creatinine',    'fasting blood sugar_minus_ALT',    'Std_Cholesterol_triglyceride',    'Cholesterol_minus_LDL',    'Std_Cholesterol_LDL',    'Cholesterol_minus_Gtp',    'triglyceride_x_AST',    'triglyceride_div_ALT',    'ALT_div_triglyceride',    'triglyceride_div_Gtp',    'HDL_x_serum creatinine',    'Std_HDL_Gtp',    'AST_div_LDL',    'Std_LDL_Gtp',    'Std_AST_ALT',    'AST_div_Gtp',    'interact_Gtp_dental caries',    'interact_fasting blood sugar_hearing(right)',    'interact_eyesight(left)_Urine protein',    'interact_HDL_Urine protein']

ridge_feats = ['ALT_risk', 'mean_eyesight(right)_by_hearing(right)', 'mean_age_by_hearing(left)', 'mean_age_by_dental caries', 'Gtp_risk', 'hemoglobin_risk', 'tot_chol_risk', 'AST_risk', 'HDL_risk', 'LDL_risk', 'BP_risk', 'triglyceride_risk', 'mean_AST_by_Urine protein', 'BMI_risk', 'mean_hemoglobin_by_Urine protein', 'mean_height(cm)_by_Urine protein', 'mean_eyesight(left)_by_Urine protein', 'age', 'height(cm)', 'weight(kg)', 'eyesight(left)', 'eyesight(right)', 'systolic', 'fasting blood sugar', 'hemoglobin', 'serum creatinine', 'AST', 'ALT', 'Gtp', 'eyesight(left)_degree2', 'eyesight(right)_degree2', 'LDL_degree2', 'serum creatinine_degree3', 'AST_degree2', 'ALT_degree2', 'Gtp_degree2', 'age_x_eyesight(left)', 'eyesight(left)_div_age', 'age_x_eyesight(right)', 'eyesight(right)_div_age', 'age_minus_systolic', 'age_minus_fasting blood sugar', 'age_x_HDL', 'age_minus_HDL', 'Std_age_HDL', 'age_div_LDL', 'age_x_serum creatinine', 'age_x_AST', 'age_minus_AST', 'Std_age_ALT', 'Std_age_Gtp', 'height(cm)_minus_waist(cm)', 'Std_height(cm)_triglyceride', 'height(cm)_div_LDL', 'height(cm)_minus_AST', 'Std_height(cm)_ALT', 'Std_height(cm)_Gtp', 'weight(kg)_minus_waist(cm)', 'Std_weight(kg)_relaxation', 'weight(kg)_x_Cholesterol', 'Std_weight(kg)_HDL', 'weight(kg)_div_serum creatinine', 'weight(kg)_minus_AST', 'Std_weight(kg)_ALT', 'Std_weight(kg)_Gtp', 'Std_waist(cm)_relaxation', 'waist(cm)_div_hemoglobin', 'eyesight(left)_x_eyesight(right)', 'eyesight(left)_minus_eyesight(right)', 'eyesight(left)_div_eyesight(right)', 'eyesight(right)_div_eyesight(left)', 'Std_eyesight(left)_eyesight(right)', 'percent_sum_eyesight(left)_eyesight(right)', 'eyesight(left)_x_triglyceride', 'eyesight(left)_div_triglyceride', 'triglyceride_div_eyesight(left)', 'eyesight(left)_div_LDL', 'Std_eyesight(left)_serum creatinine', 'eyesight(left)_x_AST', 'eyesight(left)_div_ALT', 'eyesight(left)_div_Gtp', 'Gtp_div_eyesight(left)', 'eyesight(right)_x_triglyceride', 'triglyceride_div_eyesight(right)', 'eyesight(right)_div_LDL', 'Std_eyesight(right)_serum creatinine', 'eyesight(right)_x_AST', 'eyesight(right)_div_AST', 'eyesight(right)_div_ALT', 'ALT_div_eyesight(right)', 'Gtp_div_eyesight(right)', 'systolic_minus_relaxation', 'systolic_minus_fasting blood sugar', 'Std_systolic_fasting blood sugar', 'Std_systolic_triglyceride', 'systolic_minus_HDL', 'Std_systolic_LDL', 'systolic_div_hemoglobin', 'systolic_minus_AST', 'Std_systolic_ALT', 'Std_systolic_Gtp', 'fasting blood sugar_x_Cholesterol', 'Std_fasting blood sugar_LDL', 'fasting blood sugar_x_serum creatinine', 'fasting blood sugar_div_serum creatinine', 'fasting blood sugar_minus_ALT', 'Std_fasting blood sugar_ALT', 'Std_fasting blood sugar_Gtp', 'Std_Cholesterol_triglyceride', 'Cholesterol_minus_LDL', 'LDL_div_Cholesterol', 'Std_Cholesterol_LDL', 'Cholesterol_minus_Gtp', 'triglyceride_x_AST', 'AST_div_triglyceride', 'triglyceride_div_ALT', 'ALT_div_triglyceride', 'triglyceride_div_Gtp', 'Gtp_div_triglyceride', 'HDL_x_serum creatinine', 'Std_HDL_Gtp', 'AST_div_LDL', 'ALT_div_LDL', 'Gtp_div_LDL', 'Std_LDL_Gtp', 'AST_div_ALT', 'Std_AST_ALT', 'AST_x_Gtp', 'AST_div_Gtp', 'interact_Gtp_dental caries', 'interact_weight(kg)_hearing(right)', 'interact_fasting blood sugar_hearing(right)', 'interact_Cholesterol_hearing(right)', 'interact_fasting blood sugar_hearing(left)', 'interact_Cholesterol_hearing(left)', 'interact_age_Urine protein', 'interact_eyesight(left)_Urine protein', 'interact_eyesight(right)_Urine protein', 'interact_HDL_Urine protein', 'interact_AST_Urine protein']
naive_feats = []
lda_feats = ['ALT_risk', 'mean_eyesight(right)_by_hearing(right)', 'mean_age_by_hearing(left)', 'mean_age_by_dental caries', 'Gtp_risk', 'hemoglobin_risk', 'tot_chol_risk', 'AST_risk', 'HDL_risk', 'LDL_risk', 'BP_risk', 'triglyceride_risk', 'mean_AST_by_Urine protein', 'BMI_risk', 'mean_hemoglobin_by_Urine protein', 'mean_height(cm)_by_Urine protein', 'mean_eyesight(left)_by_Urine protein', 'age', 'height(cm)', 'weight(kg)', 'eyesight(left)', 'eyesight(right)', 'systolic', 'fasting blood sugar', 'hemoglobin', 'serum creatinine', 'AST', 'ALT', 'Gtp', 'eyesight(left)_degree2', 'eyesight(right)_degree2', 'LDL_degree2', 'serum creatinine_degree3', 'AST_degree2', 'ALT_degree2', 'Gtp_degree2', 'age_x_eyesight(left)', 'eyesight(left)_div_age', 'age_x_eyesight(right)', 'eyesight(right)_div_age', 'age_minus_systolic', 'age_minus_fasting blood sugar', 'age_x_HDL', 'age_minus_HDL', 'Std_age_HDL', 'age_div_LDL', 'age_x_serum creatinine', 'age_x_AST', 'age_minus_AST', 'Std_age_ALT', 'Std_age_Gtp', 'height(cm)_minus_waist(cm)', 'Std_height(cm)_triglyceride', 'height(cm)_div_LDL', 'height(cm)_minus_AST', 'Std_height(cm)_ALT', 'Std_height(cm)_Gtp', 'weight(kg)_minus_waist(cm)', 'Std_weight(kg)_relaxation', 'weight(kg)_x_Cholesterol', 'Std_weight(kg)_HDL', 'weight(kg)_div_serum creatinine', 'weight(kg)_minus_AST', 'Std_weight(kg)_ALT', 'Std_weight(kg)_Gtp', 'Std_waist(cm)_relaxation', 'waist(cm)_div_hemoglobin', 'eyesight(left)_x_eyesight(right)', 'eyesight(left)_minus_eyesight(right)', 'eyesight(left)_div_eyesight(right)', 'eyesight(right)_div_eyesight(left)', 'Std_eyesight(left)_eyesight(right)', 'percent_sum_eyesight(left)_eyesight(right)', 'eyesight(left)_x_triglyceride', 'eyesight(left)_div_triglyceride', 'triglyceride_div_eyesight(left)', 'eyesight(left)_div_LDL', 'Std_eyesight(left)_serum creatinine', 'eyesight(left)_x_AST', 'eyesight(left)_div_ALT', 'eyesight(left)_div_Gtp', 'Gtp_div_eyesight(left)', 'eyesight(right)_x_triglyceride', 'triglyceride_div_eyesight(right)', 'eyesight(right)_div_LDL', 'Std_eyesight(right)_serum creatinine', 'eyesight(right)_x_AST', 'eyesight(right)_div_AST', 'eyesight(right)_div_ALT', 'ALT_div_eyesight(right)', 'Gtp_div_eyesight(right)', 'systolic_minus_relaxation', 'systolic_minus_fasting blood sugar', 'Std_systolic_fasting blood sugar', 'Std_systolic_triglyceride', 'systolic_minus_HDL', 'Std_systolic_LDL', 'systolic_div_hemoglobin', 'systolic_minus_AST', 'Std_systolic_ALT', 'Std_systolic_Gtp', 'fasting blood sugar_x_Cholesterol', 'Std_fasting blood sugar_LDL', 'fasting blood sugar_x_serum creatinine', 'fasting blood sugar_div_serum creatinine', 'fasting blood sugar_minus_ALT', 'Std_fasting blood sugar_ALT', 'Std_fasting blood sugar_Gtp', 'Std_Cholesterol_triglyceride', 'Cholesterol_minus_LDL', 'LDL_div_Cholesterol', 'Std_Cholesterol_LDL', 'Cholesterol_minus_Gtp', 'triglyceride_x_AST', 'AST_div_triglyceride', 'triglyceride_div_ALT', 'ALT_div_triglyceride', 'triglyceride_div_Gtp', 'Gtp_div_triglyceride', 'HDL_x_serum creatinine', 'Std_HDL_Gtp', 'AST_div_LDL', 'ALT_div_LDL', 'Gtp_div_LDL', 'Std_LDL_Gtp', 'AST_div_ALT', 'Std_AST_ALT', 'AST_x_Gtp', 'AST_div_Gtp', 'interact_Gtp_dental caries', 'interact_weight(kg)_hearing(right)', 'interact_fasting blood sugar_hearing(right)', 'interact_Cholesterol_hearing(right)', 'interact_fasting blood sugar_hearing(left)', 'interact_Cholesterol_hearing(left)', 'interact_age_Urine protein', 'interact_eyesight(left)_Urine protein', 'interact_eyesight(right)_Urine protein', 'interact_HDL_Urine protein', 'interact_AST_Urine protein']

# Cross-Validation

In [26]:
%%time

# Initialize lists for scores
hist_scores, lgbm_scores, xgb_scores, cat_scores = [], [], [], []
ada_scores, grad_scores, logreg_scores, ridge_scores = [], [], [], []
gnb_scores, lda_scores = [], []
     
for i, (train_idx, test_idx) in enumerate(sk.split(X, y)):

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    print('----------------------------------------------------------')

    ##########################
    ## HistGradientBoosting ##
    ##########################

    start_time = time.time()
    hist = HistGradientBoostingClassifier(random_state=5).fit(X_train, y_train)
    
    hist_train_pred = hist.predict_proba(X_train)[:, 1]
    hist_pred = hist.predict_proba(X_test)[:, 1]
    hist_score = roc_auc_score(y_test, hist_pred)
    hist_scores.append(hist_score)
    
    print(f'Fold {i + 1} ==> HistGradientBoosting ROC-AUC score is ==> {hist_score}')
    print(f'{time.time() - start_time}')
          
    ##########
    ## LGBM ##
    ##########

    start_time = time.time()
    lgbm = LGBMClassifier(n_jobs=-1, random_state=5).fit(X_train[lgbm_feats], y_train)

    lgbm_train_pred = lgbm.predict_proba(X_train[lgbm_feats])[:, 1]
    lgbm_pred = lgbm.predict_proba(X_test[lgbm_feats])[:, 1]
    lgbm_score = roc_auc_score(y_test, lgbm_pred)
    lgbm_scores.append(lgbm_score)
    
    print(f'Fold {i + 1} ==> LGBM ROC-AUC score is ==> {lgbm_score}')
    print(f'{time.time() - start_time}')

    #############
    ## XGBoost ##
    #############

    start_time = time.time()
    xgb = XGBClassifier(seed=5).fit(X_train, y_train)

    xgb_train_pred = xgb.predict_proba(X_train)[:, 1]
    xgb_pred = xgb.predict_proba(X_test)[:, 1]
    xgb_score = roc_auc_score(y_test, xgb_pred)
    xgb_scores.append(xgb_score)
    
    print(f'Fold {i + 1} ==> XGBoost ROC-AUC score is ==> {xgb_score}')
    print(f'{time.time() - start_time}')

    ##############
    ## CatBoost ##
    ##############

    start_time = time.time()
    cat = CatBoostClassifier(random_state=5, verbose=False, early_stopping_rounds=100).fit(X_train, y_train)

    cat_train_pred = cat.predict_proba(X_train)[:, 1]
    cat_pred = cat.predict_proba(X_test)[:, 1]
    cat_score = roc_auc_score(y_test, cat_pred)
    cat_scores.append(cat_score)
    
    print(f'Fold {i + 1} ==> CatBoost ROC-AUC score is ==> {cat_score}')
    print(f'{time.time() - start_time}')

    ########################
    ## AdaBoostClassifier ##
    ########################
    
    start_time = time.time()
    ada = AdaBoostClassifier(random_state=5).fit(X_train, y_train)
    ada_pred = ada.predict_proba(X_test)[:, 1]
    ada_score = roc_auc_score(y_test, ada_pred)
    ada_scores.append(ada_score)

    print(f'Fold {i + 1} ==> AdaBoostClassifier ROC-AUC score is ==> {ada_score}')
    print(f'{time.time() - start_time}')
    
    ################################
    ## GradientBoostingClassifier ##
    ################################
    
    start_time = time.time()
    grad = GradientBoostingClassifier(random_state=5).fit(X_train, y_train)
    grad_pred = grad.predict_proba(X_test)[:, 1]
    grad_score = roc_auc_score(y_test, grad_pred)
    grad_scores.append(grad_score)

    print(f'Fold {i + 1} ==> GradientBoostingClassifier ROC-AUC score is ==> {grad_score}')
    print(f'{time.time() - start_time}')
    
    ########################
    ## LogisticRegression ##
    ########################
    
    start_time = time.time()
    logreg = LogisticRegression(random_state=5).fit(X_train[logreg_feats], y_train)
    logreg_pred = logreg.predict_proba(X_test[logreg_feats])[:, 1]
    logreg_score = roc_auc_score(y_test, logreg_pred)
    logreg_scores.append(logreg_score)

    print(f'Fold {i + 1} ==> LogisticRegression ROC-AUC score is ==> {logreg_score}')
    print(f'{time.time() - start_time}')
    
    #####################
    ## RidgeClassifier ##
    #####################
    
    start_time = time.time()
    ridge = RidgeClassifier(random_state=5).fit(X_train[ridge_feats], y_train)
    ridge_pred = ridge.decision_function(X_test[ridge_feats])
    ridge_score = roc_auc_score(y_test, ridge_pred)
    ridge_scores.append(ridge_score)

    print(f'Fold {i + 1} ==> RidgeClassifier ROC-AUC score is ==> {ridge_score}')
    print(f'{time.time() - start_time}')
    
    ################
    ## GaussianNB ##
    ################
    
    start_time = time.time()
    gnb = GaussianNB().fit(X_train, y_train)
    gnb_pred = gnb.predict_proba(X_test)[:, 1]
    gnb_score = roc_auc_score(y_test, gnb_pred)
    gnb_scores.append(gnb_score)

    print(f'Fold {i + 1} ==> GaussianNB ROC-AUC score is ==> {gnb_score}')
    print(f'{time.time() - start_time}')
    
    ################################
    ## LinearDiscriminantAnalysis ##
    ################################
    
    start_time = time.time()
    lda = LinearDiscriminantAnalysis().fit(X_train[lda_feats], y_train)
    lda_pred = lda.predict_proba(X_test[lda_feats])[:, 1]
    lda_score = roc_auc_score(y_test, lda_pred)
    lda_scores.append(lda_score)

    print(f'Fold {i + 1} ==> LinearDiscriminantAnalysis ROC-AUC score is ==> {lda_score}')
    print(f'{time.time() - start_time}')

    print()
    
print('----------------------------------------------------------')
print(f'Average Hist Score is ==> {np.mean(hist_scores)}')
print(f'Average LGBM Score is ==> {np.mean(lgbm_scores)}')
print(f'Average XGB Score is ==> {np.mean(xgb_scores)}')
print(f'Average CAT Score is ==> {np.mean(cat_scores)}')
print(f'Average AdaBoost Score: {np.mean(ada_scores)}')
print(f'Average GradientBoosting Score: {np.mean(grad_scores)}')
print(f'Average Logistic Regression Score: {np.mean(logreg_scores)}')
print(f'Average Ridge Classifier Score: {np.mean(ridge_scores)}')
print(f'Average GaussianNB Score: {np.mean(gnb_scores)}')
print(f'Average LDA Score: {np.mean(lda_scores)}')

----------------------------------------------------------
Fold 1 ==> HistGradientBoosting ROC-AUC score is ==> 0.8575670315768426
171.77234601974487
Fold 1 ==> LGBM ROC-AUC score is ==> 0.8576791193706166
8.049715042114258


KeyboardInterrupt: 